!pip install -U sentence-transformers

In [1]:
from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity as sim
import pandas as pd

In [2]:
model_name = './nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name, device = 'cpu')

In [3]:
def get_wh_ans2(filepath, question):
    f = open(filepath, encoding = "utf-8", mode = "r")
    txt = f.read()
    txt = txt[txt.find("\n\n\n")+3:txt.find("References")]
    sentences = enumerate(sent_tokenize(txt))
    
    sentences = enumerate(sent_tokenize(txt))
    passages = [[str(index), sentence] for index, sentence in sentences]
    passage_embedding = bi_encoder.encode(passages)

    p = pd.DataFrame(passages)
    p[2] = sim(passage_embedding, q_emb.reshape(1,-1))
    p = p.sort_values(by = 2, ascending = False)
    context_sentences = list(p[1].values)

In [4]:
sentences = enumerate(sent_tokenize(txt))
passages = [[str(index), sentence] for index, sentence in sentences]
passages

[['0',
  "The Old Kingdom is the period in the third millennium (c. 2686-2181 BC) also known as the 'Age of the Pyramids' or 'Age of the Pyramid Builders' as it includes the great 4th Dynasty when King Sneferu perfected the art of pyramid building and the pyramids of Giza were constructed under the kings Khufu, Khafre, and Menkaure."],
 ['1',
  'Egypt attained its first continuous peak of civilization – the first of three so-called "Kingdom" periods (followed by the Middle Kingdom and New Kingdom) which mark the high points of civilization in the lower Nile Valley.'],
 ['2',
  'The term itself was coined by eighteenth-century historians and the distinction between the Old Kingdom and the Early Dynastic Period is not one which would have been recognized by Ancient Egyptians.'],
 ['3',
  "Not only was the last king of the Early Dynastic Period related to the first two kings of the Old Kingdom, but the 'capital', the royal residence, remained at Ineb-Hedg, the Ancient Egyptian name for Me

In [5]:
%%time
passage_embedding = bi_encoder.encode(passages)

Wall time: 3.34 s


In [6]:
question = "What was  the subject of very large works, including especially the canal to Lake Moeris around 2300 BC, which was likely also the source of water to the Giza pyramid complex centuries earlier ?"
#search(question, passage_embedding)

In [7]:
bi_encoder.encode(question).shape, passage_embedding.shape

((768,), (68, 768))

In [8]:
from sklearn.metrics.pairwise import cosine_similarity as sim

In [9]:
q_emb  = bi_encoder.encode(question)
import pandas as pd
p = pd.DataFrame(passages)
p.head()

In [10]:
import pandas as pd
p = pd.DataFrame(passages)
p.head()
p[2] = sim(passage_embedding, q_emb.reshape(1,-1))
p.sort_values(by = 2, ascending = False)

,0,1
0,0,The Old Kingdom is the period in the third mil...
1,1,Egypt attained its first continuous peak of ci...
2,2,The term itself was coined by eighteenth-centu...
3,3,Not only was the last king of the Early Dynast...
4,4,The basic justification for a separation betwe...


In [11]:
p[2] = sim(passage_embedding, q_emb.reshape(1,-1))
p.sort_values(by = 2, ascending = False)

In [12]:
p.sort_values(by = 2, ascending = False)

,0,1,2
51,51,"However, Nile flood control was still the subj...",0.830908
60,60,2649–2150 BC) was one of the most dynamic peri...,0.400029
32,32,The latter built the second pyramid and (in tr...,0.330221
15,15,Third Dynasty\n\nThe first Pharaoh of the Old ...,0.325921
29,29,"However, the full development of the pyramid s...",0.324732
...,...,...,...
48,48,Planks and the superstructure were tightly tie...,0.039507
21,21,They also perceived themselves as a specially ...,0.037992
26,26,Snefru abandoned it after the outside casing f...,0.033461
18,18,The former rulers were forced to assume the ro...,0.029428


In [ ]:
def search(query, wiki_embeddings, top_k = 10):
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, wiki_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    # Output of top-k hits
    print("Input question:", query)
    for hit in hits:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))

In [ ]:
question = "What was  the subject of very large works, including especially the canal to Lake Moeris around 2300 BC, which was likely also the source of water to the Giza pyramid complex centuries earlier ?"
#search(question, passage_embedding)

## QG - Transformer

In [ ]:
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

In [ ]:
tokenizer_t = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_t = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

In [ ]:
tokenizer_t.save_pretrained("here")

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_lg")

In [ ]:
def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer_t([input_text], return_tensors='pt')

  output = loaded_model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer_t.decode(output[0])

def generate_questions_transformer(sentences):
    qs = []
    for i in range(len(sentences)):
        doc = nlp(sentences[i])
        for ent in doc.ents:
                  if ent.label_ in ['LANGUAGE', 'NORP', 'ORG', 'PRODUCT', 'TIME', 'WORK_OF_ART']:
                      context = sentences[i]
                      qs.append([get_question(ent, context), ent])
    r_qsts = set()
    for qq in qs:
        xx = qq[0]
        r_qsts.add(xx[xx.find(":")+2:-4])
    r_qsts = list(r_qsts)
    return r_qsts

In [ ]:
from nltk import sent_tokenize
f = open("a1.txt", encoding = "utf-8", mode = "r")
txt = f.read()
txt = txt[txt.find("\n\n\n")+3:txt.find("References")]
sentences = sent_tokenize(txt)

In [ ]:
generate_questions_transformer(sentences)

In [ ]:
model_t.save_pretrained('./finetuned_qg_model_SQuAD/')

In [ ]:
loaded_model = AutoModelWithLMHead.from_pretrained('./finetuned_qg_model_SQuAD/')

# Paraphrase Mining

In [ ]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Single list of sentences - Possible tens of thousands of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

paraphrases = util.paraphrase_mining(model, sentences)

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

# Paraphrase

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")

In [ ]:
def paraphrase(text, max_length=128):

  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(input_ids=input_ids, num_return_sequences=5, num_beams=5, max_length=max_length, no_repeat_ngram_size=2, repetition_penalty=3.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds


preds = paraphrase("paraphrase: What was  the subject of very large works, including especially the canal to Lake Moerris")
for pred in preds:
  print(pred)

### Another paraphrase

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cpu' 
tokenizer_p = PegasusTokenizer.from_pretrained(model_name)
model_p = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def paraphrase(input_text,num_return_sequences = 1,num_beams = 5):
  batch = tokenizer_p([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model_p.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer_p.batch_decode(translated, skip_special_tokens=True)
  return tgt_text[0]

## Summarization

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

In [ ]:
%%time
def summarize(sentence):
    length = len(sentence.split())
    return summarizer(sentence, max_length = int(0.9 * length), min_length = int(0.7 * length), do_sample=False)[0]['summary_text']

In [ ]:
para = []
paragraphs = txt.split("\n\n")
for paragraph in paragraphs:
    if len(paragraph) > 20:
        para.append(summarize(paragraph))

In [ ]:
import claucy
import spacy

def get_triples(sentences):
    nlp3 = spacy.load('en_core_web_lg')
    claucy.add_to_pipe(nlp3)
    breakdown = []
    for sentence in sentences:
        doc = nlp3(sentence) 
        breakdown = breakdown + doc._.clauses
    return breakdown

In [ ]:
get_triples(para)

In [ ]:
import pandas as pd

In [ ]:
def string(l):
    ad = ""
    for i in l.adverbials:
        ad = ad + str(i) + " "
    a = str(l.type) + "~" +str(l.subject) + "~" + str(l.verb) + "~" + str(l.indirect_object)\
    + "~" + str(l.direct_object) \
    + "~" + str(l.complement) + "~" + ad
    return a

def return_questions_ie(sentences, ents, ents2, ents3):
    breakdown = get_triples(sentences)
    ie_data = pd.DataFrame(breakdown)[0].apply(string).str.split("~", expand = True).rename(columns={0:'type',
                                                                                          1:'subject',
                                                                                          2:'verb',
                                                                                          3:'indirect_obj',
                                                                                          4:'direct_obj',
                                                                                          5:'complement',
                                                                                          6:'adverbials'})
    ie_data = ie_data.replace("None", "")

    que = []
    for row in ie_data.iterrows():
        end = row[1][4]
        if end == "":
            end = row[1][5]
        if end == "":
            end = row[1][6]
        if end == "":
            continue
        q = "__________"+ row[1][2] + " " + row[1][3] + " " + end + " ?"
        que.append(q)
    return que

In [ ]:
return_questions_ie(para, _,_,_)

In [ ]:
para